<a href="https://colab.research.google.com/drive/1NsRNRf4SzQejjy8d04LM_AQufaqezFSg" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install VectorVault

In [ ]:
!pip install vector-vault 

<br>
<br>

# Get Your API Key Here:

### >>> [app.vectorvault.io](https://app.vectorvault.io/&aff=philosophyGPT_tutorial) <<<
*Click the link, enter your email, and an API key will be emailed to you instantly. (You will need this key to access the Vault)*

<br>

<br>

# Connect to VectorVault
Here, we create a Vault instance and set our OpenAI API key as an environment varible. 

In [ ]:
%env OPENAI_API_KEY = YOUR_OPENAI_API_KEY

In [ ]:
from vectorvault import Vault, download_url

vault = Vault(user='YOUR_EMAIL', api_key='YOUR_VECTOR_VAULT_API_KEY', vault='philosophy', verbose=True)

<br>

# Download the Philosophy Books 
1. Download the book texts from the handpicked, refinded data repo of no-copywrite philosophy texts
2. Get vectors 
3. Add to Vault

This is a large amount of text and will take roughly 9min to get the vectors and then save them to the cloud. After this is done, referencing the all of them at once will take less than a second.

*A Free account will not have the resources to fully finish this. You can stop after the first couple books, or upgrade to a [Personal account](https://buy.stripe.com/7sI03AgMldS5b7y7st)*

In [ ]:
import re
import requests 

# Get the books from the repo
response = requests.get("https://api.github.com/repos/John-Rood/Philosophy/contents/books")
files = response.json()

# Make a loop to download each book since there are 37
for file in files:
  if file["name"].endswith(".txt"):

    # Download the book text
    file_url = file["download_url"]
    text = download_url(file_url)
    print(f"Processing {file_url}")

    # Extract metadata
    title = re.search(r"Title:\s*(.+)", text)
    author = re.search(r"Author:\s*(.+)", text)

    # Handle formatting issues
    title = title.group(1).strip() if title else "Unknown Title"
    author = author.group(1).strip() if author else "Unknown Author"

    meta = {
              "title": title,
              "author": author,
            }

    # Add for processing
    vault.add(text, meta, split_size=500)

    # Get vectors for all data added
    vault.get_vectors()
    
    # Save data with vectors to the Vault Cloud
    vault.save()


# Ask Your Question

In [ ]:
user_input = input("What's your next question?: ")

# Get response from Language model
answer = vault.print_stream(vault.get_chat_stream(user_input, get_context=True))

<br>

## See where the answer is coming from with `return_context=True`
In this response, we add conversation history to to the "get_chat()" function. We also print the source context that the llm uses for the response. (Since the sources are returned along with the response, it comes in the form of a dictionary)

In [ ]:
user_input2 = input("What's your next question?: ")

history = user_input + answer

answer = vault.print_stream(vault.cloud_stream(vault.get_chat_stream(user_input2, history=history, get_context=True, return_context=True, metatag=['title', 'author'], metatag_prefixes=['\n\nTitle: ', '\nAuthor: '], metatag_suffixes=['', '\n'])))

In [ ]:
user_input3 = input("What's your next question?: ")

history = history + user_input2 + answer

response = vault.print_stream(vault.get_chat_stream(user_input3, history=history, get_context=True))

In [ ]:
# if you want to delete an api key and/or get a new one:

# import vectorvault

# response = vectorvault.delete_key('YOUR_EMAIL', 'YOUR_VECTOR_VAULT_API_KEY')
# print(response)
# response = vectorvault.get_new_key('YOUR_EMAIL', 'YOUR_PASSWORD')
# print(response)